In [ ]:
!pip install transformers datasets torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

In [ ]:
# **Step 1: Load Hindi Dataset**
file_path = ' '
df = pd.read_csv(file_path)  # CSV file ko load karein
df = df.dropna()  # Missing values hatao

# Convert text labels ("positive"/"negative") to numeric (1/0)
df['polarity'] = df['polarity'].map({"positive": 1, "negative": 0})

# Train-Test Split (80% Train, 20% Test)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'].tolist(), df['polarity'].tolist(), test_size=0.2, random_state=42)

# Hugging Face Dataset Format Convert
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

In [ ]:
# **Step 2: Load Pre-trained XLM-R Model & Tokenizer**
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 labels (Positive, Negative)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# **Step 3: Tokenization Function**
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/33345 [00:00<?, ? examples/s]

Map:   0%|          | 0/8337 [00:00<?, ? examples/s]

In [ ]:
import evaluate  # New library for metrics

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="binary")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="binary")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")

    return {
        "accuracy": acc["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }


In [ ]:
# **Step 5: Define Training Arguments**
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# **Step 6: Train the Model**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-10-4b252aabbb3f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dkk016882 (dkk016882-guru-ghasidas-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.245000,0.279403,0.926472,0.954220,0.895798,0.924087
2,0.083100,0.270305,0.941226,0.924659,0.960624,0.942299
3,0.142900,0.245388,0.950222,0.952243,0.947899,0.950066


TrainOutput(global_step=12507, training_loss=0.22146679500014094, metrics={'train_runtime': 11183.8706, 'train_samples_per_second': 8.945, 'train_steps_per_second': 1.118, 'total_flos': 2.63203144229376e+16, 'train_loss': 0.22146679500014094, 'epoch': 3.0})

In [ ]:
# **Step 7: Evaluate the Model**
print("Evaluating model on test dataset...")
eval_results = trainer.evaluate()
print(eval_results)

Evaluating model on test dataset...


{'eval_loss': 0.2453879415988922, 'eval_accuracy': 0.9502219023629603, 'eval_precision': 0.9522431259044862, 'eval_recall': 0.9478991596638655, 'eval_f1': 0.9500661773553123, 'eval_runtime': 213.2634, 'eval_samples_per_second': 39.093, 'eval_steps_per_second': 4.891, 'epoch': 3.0}


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

# Evaluate model
def compute_metrics(labels, preds):
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    report = classification_report(labels, preds, target_names=["Negative", "Positive"], digits=4)
    print("\nClassification Report:\n", report)
    print(f"\nAccuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1-score": f1}

# Run evaluation
eval_results = trainer.predict(test_dataset)
y_true = eval_results.label_ids.tolist()
y_pred = torch.argmax(torch.tensor(eval_results.predictions), dim=1).tolist()
compute_metrics(y_true, y_pred)


Classification Report:
               precision    recall  f1-score   support

    Negative     0.9482    0.9525    0.9504      4172
    Positive     0.9522    0.9479    0.9501      4165

    accuracy                         0.9502      8337
   macro avg     0.9502    0.9502    0.9502      8337
weighted avg     0.9502    0.9502    0.9502      8337


Accuracy: 0.9502
Precision: 0.9502
Recall: 0.9502
F1-Score: 0.9502


{'accuracy': 0.9502219023629603,
 'precision': 0.950231065628837,
 'recall': 0.9502219023629603,
 'f1-score': 0.9502215485694477}

In [ ]:
# **Step 8: Predict Sentiment on New Sentences**
import torch

# **Check GPU Availability & Move Model to GPU**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)  # Move model to GPU

# **Fix Predict Function**
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)  # Move input to GPU
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(-1).item()

    return "Positive" if prediction == 1 else "Negative"

Using device: cuda


In [ ]:
# **Test with Example Sentences**
test_sentences = [
    "यह फिल्म बहुत अच्छी  है!",
    "मुझे यह जगह पसंद नहीं आई।",
    "फोन की बैटरी लाइफ बहुत खराब है।",
    "यह प्रोडक्ट बहुत शानदार है!"
]

for sentence in test_sentences:
    print(f"Text: {sentence} --> Sentiment: {predict_sentiment(sentence)}")

Text: यह फिल्म बहुत अच्छी  है! --> Sentiment: Positive
Text: मुझे यह जगह पसंद नहीं आई। --> Sentiment: Negative
Text: फोन की बैटरी लाइफ बहुत खराब है। --> Sentiment: Negative
Text: यह प्रोडक्ट बहुत शानदार है! --> Sentiment: Positive
